In [1]:
import sys
import os
import numpy as np
import opensn

In [2]:
from mpi4py import MPI
size = MPI.COMM_WORLD.size
rank = MPI.COMM_WORLD.rank

from pyopensn.mesh import FromFileMeshGenerator, PETScGraphPartitioner
from pyopensn.xs import MultiGroupXS
from pyopensn.aquad import GLCProductQuadrature3DXYZ
from pyopensn.solver import DiscreteOrdinatesProblem, NonLinearKEigenSolver
from pyopensn.fieldfunc import FieldFunctionGridBased

In [3]:
meshgen = FromFileMeshGenerator(filename="case_6.msh", partitioner=PETScGraphPartitioner(type='parmetis'))
grid = meshgen.Execute()

OpenSn version 0.0.1
2025-05-05 20:11:06 Running OpenSn with 1 processes.

[0]  FromFileMeshGenerator: Generating UnpartitionedMesh
[0]  Making unpartitioned mesh from Gmsh file case_6.msh (format v4.1)
[0]  Mesh identified as 3D.
[0]  Done checking cell-center-to-face orientations
[0]  00:00:00.0 Establishing cell connectivity.
[0]  00:00:00.1 Vertex cell subscriptions complete.
[0]  00:00:00.1 Surpassing cell 1439 of 14383 (10%)
[0]  00:00:00.1 Surpassing cell 2877 of 14383 (20%)
[0]  00:00:00.2 Surpassing cell 4315 of 14383 (30%)
[0]  00:00:00.2 Surpassing cell 5754 of 14383 (40%)
[0]  00:00:00.2 Surpassing cell 7192 of 14383 (50%)
[0]  00:00:00.2 Surpassing cell 8630 of 14383 (60%)
[0]  00:00:00.2 Surpassing cell 10069 of 14383 (70%)
[0]  00:00:00.3 Surpassing cell 11507 of 14383 (80%)
[0]  00:00:00.3 Surpassing cell 12945 of 14383 (90%)
[0]  00:00:00.3 Surpassing cell 14383 of 14383 (100%)
[0]  00:00:00.3 Establishing cell boundary connectivity.
[0]  00:00:00.3 Done establishing c

In [4]:
volumes_per_block = grid.ComputeVolumePerBlockID()

In [5]:
radii = {1: 6.0509, 2: 26.3709}
block_ids = sorted(volumes_per_block.keys())

In [6]:
exact_volumes_per_block = {}
prev_R = 0.0
for blk in block_ids:
    R = radii[blk]
    # shell from prev_R to R
    exact_volumes_per_block[blk] = (4.0 / 3.0) * np.pi * (R**3 - prev_R**3)
    prev_R = R

In [7]:
ratios = np.array([
    exact_volumes_per_block[blk] / volumes_per_block[blk]
    for blk in block_ids
])


In [8]:
if rank == 0:
    for idx, blk in enumerate(block_ids):
        print(f"Block {blk}: measured = {volumes_per_block[blk]:.11e}, "
              f" exact = {exact_volumes_per_block[blk]:.11e}, "
              f" ratio = {ratios[idx]:.6f}")


Block 1: measured = 8.61216846618e+02,  exact = 9.28001196605e+02,  ratio = 1.077546
Block 2: measured = 7.56068177582e+04,  exact = 7.58900817710e+04,  ratio = 1.003747


In [9]:
xs_oralloy = MultiGroupXS()
xs_oralloy.LoadFromOpenMC("oralloy.h5", "set1", 294.0)
xs_oralloy.SetScalingFactor(ratios[0])

xs_tuballoy = MultiGroupXS()
xs_tuballoy.LoadFromOpenMC("tuballoy.h5", "set1", 294.0)
xs_tuballoy.SetScalingFactor(ratios[1])

num_groups = xs_tuballoy.num_groups
if rank == 0:
    print("num groups =", num_groups)

num groups =[0]  Reading OpenMC cross-section file "oralloy.h5"
 30
[0]  oralloy.h5 cross-section data evaluated at 294K
[0]  *** WARNING ***  Estimating absorption from the transfer matrices.
[0]  *** WARNING ***  Negative transport cross section found for group 27 in call to ComputeDiffusionParameters. sigma_t=0 sigma_1=0. Setting sigma_1=0, sigma_tr=sigma_t for this group.
[0]  *** WARNING ***  Negative transport cross section found for group 28 in call to ComputeDiffusionParameters. sigma_t=0 sigma_1=0. Setting sigma_1=0, sigma_tr=sigma_t for this group.
[0]  *** WARNING ***  Negative transport cross section found for group 29 in call to ComputeDiffusionParameters. sigma_t=0 sigma_1=0. Setting sigma_1=0, sigma_tr=sigma_t for this group.
[0]  *** WARNING ***  Negative transport cross section found for group 27 in call to ComputeDiffusionParameters. sigma_t=0 sigma_1=0. Setting sigma_1=0, sigma_tr=sigma_t for this group.
[0]  *** WARNING ***  Negative transport cross section found fo

In [10]:
phys = DiscreteOrdinatesProblem(
    mesh=grid,
    num_groups=num_groups,
    groupsets=[
        {
            "groups_from_to": (0, num_groups - 1),
            "angular_quadrature": GLCProductQuadrature3DXYZ(8, 16),
            "inner_linear_method": "petsc_gmres",
            "angle_aggregation_type": "single",
            "angle_aggregation_num_subsets": 1,
            "l_max_its": 500,
            "l_abs_tol": 1.0e-6,
        },
    ],
    xs_map=[
        {"block_ids": [1], "xs": xs_oralloy},
        {"block_ids": [2], "xs": xs_tuballoy},
    ],
    options={
        "scattering_order": 3,
        "use_precursors": False,
        "verbose_inner_iterations": True,
        "verbose_outer_iterations": True,
    },
)


In [ ]:
k_solver = NonLinearKEigenSolver(
    lbs_problem=phys,
    nl_max_its=500,
    nl_abs_tol=1.0e-8,
)
k_solver.Initialize()
k_solver.Execute()
k = k_solver.GetEigenvalue()
# only rank 0 prints
if rank == 0:
    print(f"Computed k-eigenvalue: {k}")


In [12]:
fflist = phys.GetScalarFieldFunctionList(only_scalar_flux=False)
vtk_basename = "HEU_MET_FAST_003"
# export only the flux of group g (first []), moment 0 (second [])
FieldFunctionGridBased.ExportMultipleToVTK(
    [fflist[g][0] for g in range(num_groups)],
    vtk_basename
)

[0]  Exporting field functions to VTK with file base "HEU_MET_FAST_003"
[0]  Done exporting field functions to VTK.
